In [1]:
from cim.loaders import Parameter, ConnectionParameters
from cim.loaders.blazegraph import BlazegraphConnection
from cim.loaders.gridappsd import GridappsdConnection, get_topology_response
from cim.models.distributed_model import DistributedModel
import cim.data_profile as cim
import json


In [2]:
# Blazegraph connection for running inside the container
params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [3]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [4]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
#feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [5]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [6]:
topology_response = get_topology_response(feeder_mrid)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


In [7]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

LinearShuntCompensator
PowerTransformer
LoadBreakSwitch
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch


In [8]:
network.switch_areas[0].get_all_attributes(cim.LinearShuntCompensator)
network.switch_areas[0].get_all_attributes(cim.ACLineSegment)
network.switch_areas[0].get_all_attributes(cim.WireSpacingInfo)




        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT ?mRID ?name ?BaseVoltage ?Location ?RegulatingControl ?aVRDelay ?b0PerSection 
        ?bPerSection ?controlEnabled ?g0PerSection ?gPerSection ?grounded ?maximumSections ?nomU 
        ?normalSections ?phaseConnection ?sections ?Terminals ?ShuntCompensatorPhase 
        (group_concat(distinct ?Measurement; separator=";") as ?Measurements) 
        WHERE { 
          ?eq r:type cim:LinearShuntCompensator.
          VALUES ?fdrid {"_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"}
          VALUES ?mRID { "_6A7D7AF8-A0F7-41B6-BB0C-160EAE054DEA" 
               } 
        ?eq cim:Equipment.EquipmentContainer ?fdr. #filter by feeder
        ?fdr cim:IdentifiedObject.mRID ?fdrid. #filter by feeder
        ?eq cim:IdentifiedObject.mRID ?mRID. #get cap mrid
        ?eq cim:IdentifiedObject.name ?name. #get cap name

        ?eq cim:ConductingEquipment.BaseVoltage ?bv. #get 

In [9]:
network.switch_areas[1].secondary_areas[0].get_all_attributes(cim.EnergyConsumer)
network.switch_areas[0].secondary_areas[0].get_all_attributes(cim.EnergyConsumerPhase)

IndexError: list index out of range

In [ ]:
network.switch_areas[0].secondary_areas[0].typed_catalog[cim.EnergyConsumerPhase]

In [ ]:
bg.create_default_instances(feeder.mRID, ['_48A5C17E-B098-446D-B26F-2D1B198ED260'] )

In [10]:
network.get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.WireSpacingInfo)
network.get_all_attributes(cim.WirePosition)


        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT ?mRID ?name ?BaseVoltage ?Location ?length ?bch ?r ?x ?gch ?b0ch ?r0 ?x0 ?g0ch 
        ?PerLengthImpedance ?WireSpacingInfo
        (group_concat(distinct ?Terminal; separator=";") as ?Terminals) 
        (group_concat(distinct ?Measurement; separator=";") as ?Measurements) 
        (group_concat(distinct ?ACLineSegmentPhase; separator=';') as ?ACLineSegmentPhases)  
        WHERE {          
          ?eq r:type cim:ACLineSegment.
          VALUES ?fdrid {"_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"}
          VALUES ?mRID { "_08848DC6-BB11-4647-9C21-475BAD66A8B8" 
 "_173879C4-4FDB-4521-A60D-5B102B3781B2" 
 "_4774208A-B9AB-4051-8BC9-4F7C8EA6552A" 
 "_603106B5-1DC7-40B1-A264-98774F6FD16D" 
 "_67A2574B-8146-464D-B93C-B9FCBCE913BC" 
 "_6DEF3353-8276-402F-AC8E-3DEF4A396FFE" 
 "_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB" 
 "_8A62F620-7C58-4662-AE5C-F41F5E3F0022" 
 "_8C5

In [51]:
a

WireSpacingInfo(mRID='_E804FC3A-4DE8-4497-B83D-EBE8974C791B', aliasName=None, description=None, name='602', Names=[], Assets=[], PowerSystemResources=[], isCable='false', phaseWireCount='1', phaseWireSpacing='0', ACLineSegments='_67A2574B-8146-464D-B93C-B9FCBCE913BC', WireAssemblyInfo=[], WirePositions=['_4646B654-2416-424E-B953-6D81AFA95358', '_62281BF1-8A90-4E20-B4D3-F2C679E8C20C', '_85BA2D54-B38B-458E-8C9C-CEB7D2BF8FD8', '_FDE19188-F0F1-4EFC-B0F1-FFE9391ADE02'])

In [82]:
from typing import List
import re
a = network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B']
at_type = a.__dataclass_fields__['ACLineSegments'].type
print(at_type)
if 'List' in at_type:
    at_cls = re.match(r'List\[\'(.*)\']',at_type)
    print(at_cls.group(1))
    # re.match(r'<img.*?>',line)
# imgtag.group(0)


List['ACLineSegment']
ACLineSegment


In [32]:
#ridAPPSD Agent Side
bg.create_default_instances(feeder.mRID, network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B'].WirePositions)
network.get_all_attributes(cim.WirePosition)
network.typed_catalog[cim.ACLineSegment]

WirePosition
WirePosition
WirePosition
WirePosition


{'_08848DC6-BB11-4647-9C21-475BAD66A8B8': ACLineSegment(mRID='_08848DC6-BB11-4647-9C21-475BAD66A8B8', aliasName=None, description=None, name='671684', Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=Location(mRID='_2DE7691B-723B-416A-87B6-35B22FBA52B6', aliasName=None, description=None, name=None, Names=[], Assets=[], CoordinateSystem=None, Fault=[], Measurements=[], PowerSystemResources=[]), Measurements=['_6bf90aff-8a2e-431f-b159-2df722dee9f4', '_a190d1ea-47a7-482a-83ba-ed1936917bee', '_09c6dcc8-20d9-4249-bf99-efc6a4219812', '_212713f5-2b03-4282-993c-3064558680fa'], inService=None, Faults=[], OperationalLimitSet=[], BaseVoltage=BaseVoltage(mRID='_3CB8BE1F-37D8-4CC8-88BB-5C0D14D7D7B3', aliasName=None, description=None, name=None, Names=[], nominalVoltage=None, ConductingEquipment=[], TopologicalNode=[], TransformerEnds=[]), SvStatus=[], Terminals=['_636377F0-4E50-4C65-9320-35E0A68731BE', '_38B542B8-1A65-4211-9B2C-73FC39382959'], length='91.44', b0ch=None, bch=None, g0c

In [30]:
json.loads(network.switch_areas[0].__dumps__(cim.ACLineSegment))


{'_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB': {'mRID': '_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB',
  'aliasName': '',
  'description': '',
  'name': '692675',
  'Names': [],
  'AssetDatasheet': '',
  'Assets': [],
  'Controls': [],
  'Location': {'mRID': '_6D36746B-C083-4CE2-87DD-256D6C668613',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'Assets': [],
   'CoordinateSystem': None,
   'Fault': [],
   'Measurements': [],
   'PowerSystemResources': []},
  'Measurements': ['_18b33a13-c766-48f2-af7a-09027fd38693',
   '_4e706e09-bcd3-4177-8936-8981552cfd0f',
   '_6fa6beec-44fd-487c-b8ec-592c5e2e7ff4'],
  'inService': '',
  'Faults': [],
  'OperationalLimitSet': [],
  'BaseVoltage': {'mRID': '_3CB8BE1F-37D8-4CC8-88BB-5C0D14D7D7B3',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'nominalVoltage': None,
   'ConductingEquipment': [],
   'TopologicalNode': [],
   'TransformerEnds': []},
  'SvStatus': [],
  'Terminals': ['_30DE3992-1

In [ ]:
# App Side
# contextmanager.typed_catalog[cim.ACLineSegment]

In [ ]:
json.loads(network.switch_areas[0].secondary_areas[0].__dumps__(cim.EnergyConsumerPhase))

In [37]:
attribute = 'ACLineSegmentPhases'
at_type = cim.ACLineSegment().__annotations__[attribute]
at_type.split


['List', 'ACLineSegmentPhase]']

In [38]:
.__annotations__

{'b0ch': 'Optional[float]',
 'bch': 'Optional[float]',
 'g0ch': 'Optional[float]',
 'gch': 'Optional[float]',
 'r': 'Optional[float]',
 'r0': 'Optional[float]',
 'x': 'Optional[float]',
 'x0': 'Optional[float]',
 'ACLineSegmentPhases': 'List[ACLineSegmentPhase]',
 'PerLengthImpedance': 'Optional[PerLengthImpedance]',
 'WireSpacingInfo': 'Optional[WireSpacingInfo]'}